# Solução do problema do Caixeiro Viajante (TSP)

### Uso de três técnicas diferentes
1. Busca exaustiva
2. Hill Climbing
3. Simulated Annealing

In [ ]:
# Stephen Marsland, 2008, 2014
# Adaptação e correção de bugs por Hendrik Macedo

import numpy as np
import time

def makeTSP(nCities):
	positions = 2*np.random.rand(nCities,2)-1;
	distances = np.zeros((nCities,nCities))

	for i in range(nCities):
		for j in range(i+1,nCities):
			distances[i,j] = np.sqrt((positions[i,0] - positions[j,0])**2 + (positions[i,1] - positions[j,1])**2);
			distances[j,i] = distances[i,j];
	# distances[0,0] = 0
	# distances[0,1] = 1
	# distances[0,2] = 2
	# distances[0,3] = 3
	# distances[0,4] = 4
	# distances[1,1] = 0
	# distances[1,2] = 5
	# distances[1,3] = 6
	# distances[1,4] = 7
	# distances[2,2] = 0
	# distances[2,3] = 8
	# distances[2,4] = 9
	# distances[3,3] = 0
	# distances[3,4] = 1
	# distances[4,4] = 0
	# for i in range(nCities):
	# 	for j in range(i+1,nCities):
	# 		distances[j,i] = distances[i,j];

	return distances

def permutation(order):
	order = tuple(order)
	if len(order)==1:
		yield order
	else:
		for i in range(len(order)):
			rest = order[:i] + order[i+1:]
			move = (order[i],)
			for smaller in permutation(rest):
				yield move + smaller

def parameters(nCities):
	distances = makeTSP(nCities)
	return distances

In [ ]:
def exhaustive(distances):
	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],cityOrder[0]]

	for newOrder in permutation(range(nCities)):
		possibleDistanceTravelled = 0
		for i in range(nCities-1):
			possibleDistanceTravelled += distances[newOrder[i],newOrder[i+1]]
		possibleDistanceTravelled += distances[newOrder[nCities-1],newOrder[0]]
				
		if possibleDistanceTravelled < distanceTravelled:
			distanceTravelled = possibleDistanceTravelled
			cityOrder = newOrder

	return cityOrder, distanceTravelled

In [ ]:
def hillClimbing(distances):

  nCities = np.shape(distances)[0]

  cityOrder = np.arange(nCities)
  np.random.shuffle(cityOrder)

  distanceTravelled = 0
  for i in range(nCities-1):
    distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
  distanceTravelled += distances[cityOrder[nCities-1],cityOrder[0]]

  while True:

    neighbors = []
    for i in range(nCities-1):
      for j in range(nCities):
        neighbor = cityOrder.copy()
        neighbor[i] = cityOrder[j]
        neighbor[j] = cityOrder[i]
        neighbors.append(neighbor)

    neighbors = np.array(neighbors)

    best = []
    for i in range(len(neighbors)):
      newDistanceTravelled = 0
      for j in range(nCities-1):
        newDistanceTravelled += distances[neighbors[i][j],neighbors[i][j+1]]
      newDistanceTravelled += distances[neighbors[i][nCities-1],neighbors[i][0]]
      if newDistanceTravelled < distanceTravelled:
        best = neighbors[i]
        distanceTravelled = newDistanceTravelled
          
    if len(best) == 0:  
      break
    cityOrder = best

  return cityOrder, distanceTravelled

In [ ]:
def simulatedAnnealing(distances):

  nCities = np.shape(distances)[0]

  cityOrder = np.arange(nCities)
  np.random.shuffle(cityOrder)

  distanceTravelled = 0
  for i in range(nCities-1):
    distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
  distanceTravelled += distances[cityOrder[nCities-1],cityOrder[0]]

  T = 1000000
  c = 0.95

  while T>1:
  #for i in range(time):
    # if T < 1:
    #   return cityOrder, distanceTravelled
    # Choose cities to swap
    city1 = np.random.randint(nCities)
    city2 = np.random.randint(nCities)

    if city1 != city2:
      # Reorder the set of cities
      possibleCityOrder = cityOrder.copy()
      possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
      possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
      possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

      # Work out the new distances
      # This can be done more efficiently
      newDistanceTravelled = 0
      for j in range(nCities-1):
        newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
      newDistanceTravelled += distances[possibleCityOrder[nCities-1],possibleCityOrder[0]]  
      rnd = np.random.rand()
      Tln = T*np.log(rnd)
      dE = distanceTravelled - newDistanceTravelled
      if (dE >= 0) or (dE > Tln):
        distanceTravelled = newDistanceTravelled
        cityOrder = possibleCityOrder

      # Annealing schedule
      T = c*T

  return cityOrder, distanceTravelled

In [ ]:
def runExaustive(distances):
	print ("Busca exaustiva")
	start = time.time()
	result = exhaustive(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runHill(distances):
	print ("\nHill Climbing")
	start = time.time()
	result = hillClimbing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runSimAnnealing(distances):
	print ("\nSimulated Annealing")
	start = time.time()
	result = simulatedAnnealing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

#runAll()

In [ ]:
n = 9
distances = parameters(n)
print(distances)

[[0.         1.40892178 1.03751508 2.14518012 2.04942364 1.18092471
  1.05695143 2.35950869 1.57011862 1.27777732]
 [1.40892178 0.         0.37657206 1.14425209 1.25235359 0.96320685
  0.52173719 1.18442561 0.16740816 0.14416597]
 [1.03751508 0.37657206 0.         1.29826707 1.32287752 0.76336436
  0.38403897 1.42324569 0.53328008 0.24038935]
 [2.14518012 1.14425209 1.29826707 0.         0.29879056 1.0328802
  1.62666955 0.34339978 1.04339266 1.15486915]
 [2.04942364 1.25235359 1.32287752 0.29879056 0.         0.88177785
  1.68445264 0.63833013 1.18547006 1.22865262]
 [1.18092471 0.96320685 0.76336436 1.0328802  0.88177785 0.
  1.13576494 1.31591144 1.02594775 0.84547715]
 [1.05695143 0.52173719 0.38403897 1.62666955 1.68445264 1.13576494
  0.         1.70055898 0.67934396 0.47264625]
 [2.35950869 1.18442561 1.42324569 0.34339978 0.63833013 1.31591144
  1.70055898 0.         1.04743069 1.23539455]
 [1.57011862 0.16740816 0.53328008 1.04339266 1.18547006 1.02594775
  0.67934396 1.047430

In [ ]:
runExaustive(distances)

Busca exaustiva
Ordem: (0, 6, 2, 9, 1, 8, 7, 3, 4, 5)  Distancia: 5.745277465267696
Tempo: 45.7928900718689


In [ ]:
runHill(distances)


Hill Climbing
Ordem: [3 7 8 1 9 2 6 0 5 4]  Distancia: 5.745277465267697
Tempo: 0.013528108596801758


In [ ]:
runSimAnnealing(distances)


Simulated Annealing
Ordem: [5 0 6 7 3 2 1 9 4 8]  Distancia: 9.54091043171774
Tempo: 0.016772031784057617
